In [52]:
import numpy as np
import pandas as pd
import os 

In [53]:
path="C:\\Workplace\\C#\\Test\\immonium_mass_neh\\immonium_mass\\bin\\x64\\Debug\\"
data=pd.DataFrame()

for f in ["Liver_l0.csv"]:
    
    data=pd.concat([data,pd.read_csv(path+f)])
    
    
data["ion_count"]=[ len(str(x).split('|')) for x in data[["values"]].values]
print(data.shape)
data=data[data.ion_count>=2]
data=data.reset_index(drop=True)
print(data.shape)

(26187, 8)
(15339, 8)


In [54]:
res=[]
peps=[]
count=0
for i in range(data.shape[0]): 
    temp=[ int(float(x)*100)/100 for x in data["values"][i].split("|")]
    res.append(temp)
    peps.append((data["peptide"][i]))
    count+=1
print(count)

15339


In [55]:
df=pd.DataFrame()
df["pep"]=peps
df["vals"]=res

In [56]:
print(len(peps))
pd.DataFrame(np.unique([j for i in [list(x) for x in peps] for j in i],return_counts=True))

15339


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y
1,15181,2516,10153,12985,8104,14075,4934,10733,8913,19469,4274,7351,9257,8218,7521,12165,10404,14170,2214,5857


In [57]:
df_mzs=pd.DataFrame(np.unique([j for i in df.vals for j in i],return_counts=True)).transpose()
df_mzs.columns=["mz","count_s"]
df_mzs=df_mzs[df_mzs.count_s>100]
df_mzs=df_mzs.sort_values(by="mz").reset_index(drop=True)
df_mzs.head()

,mz,count_s
0,65.39,197.0
1,69.07,384.0
2,70.02,137.0
3,70.06,1475.0
4,71.06,233.0


In [58]:
mz_dict={mz:{"aas":[],"count":0} for mz in df_mzs.mz}
print(df.shape[0])
_mzs=set(df_mzs.mz)
for i in range(df.shape[0]):
#     print(i)
#     temp_mz_vals=[x for x in df["vals"][i] if x in list(df_mzs.mz)]
    
    temp_mz_vals=set(df["vals"][i]).intersection(_mzs)
        
    for mz_val in temp_mz_vals:
        mz_dict[mz_val]["aas"]+= list(np.unique(list(df["pep"][i])))
        mz_dict[mz_val]["count"]+=1 
    
    if i%10000==0:
        print(i)

15339
0
10000


In [62]:
# print(mz_dict[70.0]['count'])
# print(mz_dict[70.0]['aas'].count('W'))
# # np.unique(mz_dict[82.91]['aas'],return_counts=True)

In [63]:
aas=(['A' ,'C', 'D' ,'E', 'F', 'G', 'H', 'I' ,'K', 'L', 'M', 'N' ,'P' ,'Q' ,'R' ,'S', 'T', 'V', 'W' ,'Y'])

In [64]:
res=[]
for key in mz_dict.keys():
    _temp_count=mz_dict[key]['count']
    _temp_aas=mz_dict[key]['aas']
    
    _temp_res=[key,_temp_count] + [_temp_aas.count(c)/_temp_count for c in aas]
#     _temp_res=[key,_temp_count] + [x if x >0.7 else 0 for x in [_temp_aas.count(c)/_temp_count for c in aas]]
    res.append(_temp_res)    
    print(key)

65.39
69.07
70.02
70.06
71.06
72.08
72.09
72.1
73.08
74.06
75.45
79.88
79.89
80.63
80.64
80.65
80.66
80.67
80.7
80.71
80.72
80.8
80.81
80.83
80.84
80.85
80.86
80.87
80.88
80.89
80.93
81.06
83.06
84.04
84.07
84.08
85.04
85.08
86.05
86.06
86.09
86.1
86.14
87.05
87.1
88.03
88.04
89.0
89.01
89.02
89.07
91.05
93.04
94.17
98.02
98.06
98.98
99.05
99.78
99.79
99.8
100.07
101.06
101.07
101.1
102.05
103.05
104.04
104.05
105.05
105.06
107.04
110.06
110.07
111.07
112.07
112.08
112.64
112.65
112.66
112.67
113.07
114.09
115.05
115.08
116.01
116.07
117.06
117.1
119.04
119.05
119.08
120.04
120.07
120.08
121.08
122.07
124.03
124.11
125.07
125.1
126.05
126.09
127.05
127.08
128.08
128.1
128.16
128.17
128.18
128.19
129.06
129.09
129.1
129.11


In [68]:
res=pd.DataFrame(res)
res.columns=["mz","count"]+sorted([x for x in (aas)])
res.to_csv("test_Liver_l0.csv",index=False)

In [69]:
res.head(20)

,mz,count,A,C,D,E,F,G,H,I,...,M,N,P,Q,R,S,T,V,W,Y
0,65.39,185,0.589189,0.048649,0.194595,0.291892,0.427027,0.464865,0.124324,0.362162,...,0.183784,0.167568,0.313514,0.183784,0.454054,0.275676,0.291892,0.513514,0.086486,0.167568
1,69.07,383,0.509138,0.026110,0.219321,0.302872,0.357702,0.469974,0.093995,0.718016,...,0.159269,0.211488,0.313316,0.151436,0.381201,0.321149,0.305483,0.485640,0.057441,0.151436
2,70.02,135,0.488889,0.051852,0.377778,0.362963,0.400000,0.400000,0.081481,0.325926,...,0.155556,0.414815,0.311111,0.185185,0.444444,0.451852,0.259259,0.451852,0.125926,0.229630
3,70.06,1265,0.530435,0.060870,0.279842,0.324111,0.359684,0.456917,0.130435,0.427668,...,0.196838,0.203162,0.332806,0.200000,0.484585,0.340711,0.313043,0.524901,0.075889,0.201581
4,71.06,233,0.484979,0.051502,0.253219,0.274678,0.321888,0.399142,0.077253,0.420601,...,0.141631,0.197425,0.613734,0.184549,0.476395,0.373391,0.309013,0.515021,0.103004,0.218884
5,72.08,1523,0.508864,0.068943,0.300722,0.349311,0.356533,0.460932,0.127380,0.429416,...,0.195666,0.211425,0.318450,0.208798,0.472751,0.360473,0.342088,0.552200,0.077479,0.208798
6,72.09,447,0.536913,0.046980,0.295302,0.342282,0.420582,0.467562,0.102908,0.456376,...,0.203579,0.210291,0.333333,0.178971,0.445190,0.326622,0.355705,0.548098,0.105145,0.174497
7,72.10,601,0.499168,0.071547,0.282862,0.364393,0.314476,0.460899,0.156406,0.404326,...,0.176373,0.201331,0.349418,0.214642,0.472546,0.399334,0.344426,0.510815,0.068220,0.232945
8,73.08,267,0.483146,0.048689,0.318352,0.299625,0.382022,0.438202,0.067416,0.397004,...,0.157303,0.209738,0.322097,0.205993,0.426966,0.314607,0.348315,0.775281,0.052434,0.172285
9,74.06,1085,0.507834,0.073733,0.282949,0.346544,0.341935,0.461751,0.140092,0.402765,...,0.206452,0.212903,0.309677,0.218433,0.495853,0.378802,0.482949,0.519816,0.073733,0.235023


In [70]:
net_file=[]
for i in range(res.shape[0]):
    _temp_mz=res.mz[i]
    for aa in sorted([x for x in (aas)]):
        net_file.append([_temp_mz,aa,res[aa][i]/100])
    

In [42]:
# pd.DataFrame(net_file).to_csv("net_test.txt",index=False)

In [14]:
data=pd.read_csv("Liver_l0.csv")
data["ion_count"]=[ len(str(x).split('|')) for x in data[["values"]].values]

In [15]:
print(data.shape)
data=data[data.ion_count>=2]
data=data.reset_index(drop=True)
print(data.shape)

(26187, 8)
(15339, 8)


In [16]:
res=[]
peps=[]
count=0
for i in range(data.shape[0]): 
    temp=[ int(float(x)*100)/100 for x in data["values"][i].split("|")]
    res.append(temp)
    peps.append((data["peptide"][i]))
    count+=1
print(count)

15339


In [17]:
_test=[]
for i in res:
    _test+=i
    
_test=pd.DataFrame(np.unique(_test,return_counts=True)).transpose()
_test.columns=['x','c']
_test=_test.sort_values(by='c',ascending=False)
mzs=list(_test.head(40).x)

In [18]:
df=pd.DataFrame()
df["pep"]=peps
df["vals"]=res

In [19]:
data_dict={}

for i in range(df.shape[0]):
    
    print(df.pep[i])
    for aa in list(np.unique([x for x in df.pep[i]])):
        if aa in data_dict.keys(): 
            data_dict[aa]["vals"]=data_dict[aa]["vals"]+[ x for x in list( list(np.unique([x for x in df.vals[i]])) ) if x in mzs]
            data_dict[aa]["count"]+=1
            
            if (aa=='P'):
                print ([ x for x in list( list(np.unique([x for x in df.vals[i]])) ) if x in mzs])
            
        else:
            data_dict[aa]={"vals":list(np.unique([x for x in df.vals[i]])),"count":1}
            
            if (aa=='P'):
                print ([ x for x in list( list(np.unique([x for x in df.vals[i]])) ) if x in mzs])
            
    if i >100: break
#     for v in vals:
#         if v in data_dict.keys():
#             data_dict[v]["aa"]=data_dict[v]["aa"]+list( list(np.unique([x for x in df.pep[i]])) )
#             data_dict[v]["count"]+=1
#         else:
#             data_dict[v]={"aa":list(np.unique([x for x in df.pep[i]])),"count":1}

DADPILR
[70.06, 79.89, 84.08, 86.09, 87.05, 89.01, 99.79, 120.08, 129.1]
CGVELFK
AMLSTGFK
AMLSTGFK
LFAEAVQK
LFAEAVQK
IITGLAAGAK
NLAVPLYK
[70.06, 72.08, 79.88, 84.08, 86.09, 87.05, 87.1, 89.01, 99.79, 112.65, 120.08, 128.17, 129.1]
NLAVPLYK
[70.06, 72.08, 79.88, 84.04, 84.08, 86.09, 87.05, 87.1, 89.01, 99.79, 101.1, 110.07, 112.65, 120.08, 128.17, 129.1]
YMESDGIK
VVAVDCGIK
VVAVDCGIK
MSLDEIVR
NCGVELFK
TNQILVER
EVEMDAVGK
EVEMDAVGK
ETLIDLGTK
APMFSWPR
[70.06, 72.08, 79.88, 86.09, 89.01, 99.79, 104.05, 110.07, 112.65, 115.08, 120.08]
APMFSWPR
[70.06, 99.78, 104.05, 112.08, 120.08, 128.17]
APMFSWPR
[70.06, 72.08, 79.89, 84.04, 84.08, 86.09, 87.05, 89.01, 99.79, 101.07, 102.05, 104.05, 110.07, 112.08, 112.66, 115.08, 120.08, 121.08, 128.18, 129.1]
EVEMDAVGK
HLGIVGECN
TFEESFQK
TFEESFQK
TFEESFQK
GTTITSVLPK
[72.08, 74.06, 79.88, 84.04, 84.08, 86.09, 99.78, 102.05, 112.65, 113.07, 129.1]
GTTITSVLPK
[74.06, 79.88, 84.04, 84.08, 86.09, 99.78, 112.65, 113.07, 128.17, 129.1]
SVGEVMAIGR
KNIHPWVK
[72.08

In [20]:
# aas=sorted(['A' ,'C', 'D' ,'E', 'F', 'G', 'H', 'I' ,'K', 'L', 'M', 'N' ,'P' ,'Q' ,'R' ,'S', 'T', 'V', 'W' ,'Y'])
_index=[]
vals=[]

for k in data_dict.keys():
    _count=data_dict[k]["count"]
    if(_count>1000):
        temp_count=[]
        for c in mzs:
            temp_count.append(100*data_dict[k]["vals"].count(c)/_count)
        vals.append(temp_count)
        _index.append(k)
    break

In [21]:
xx=pd.DataFrame(np.unique(data_dict["P"]['vals'],return_counts=True)).transpose()
xx[xx.iloc[:,0]==70.06]

# data_dict.keys()

,0,1
7,70.06,6.0


In [22]:
len(vals)

0

In [23]:
_vals=pd.DataFrame(vals)
_vals.columns=mzs
# _vals["mz"]=_index
_vals.index=_index
_vals=_vals.sort_index()
# _vals=_vals.sort_values(by="mz")
_vals.transpose().to_csv("aa_percentage2.csv")

ValueError: Length mismatch: Expected axis has 0 elements, new values have 40 elements

In [ ]:
_vals

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
plt.figure(figsize=(40,20))
dataplot = sns.heatmap(_vals.round(2), cmap="YlGnBu", annot=True)
plt.savefig("_vals.jpeg",dpx=900)#,bbox_inches='tight'
plt.show()

In [ ]:
_index=[]
_seq=[]
_per=[]
for k in data_dict.keys():
    if(len(data_dict[k])>1000):
        print(k,"\t ==> \t",len(data_dict[k]))
        x,y=(np.unique(data_dict[k],return_counts=True))
        y=[ int(10000*i/sum(y))/100 for i in y]
#         print(x,y)
        
        temp=set(['A' ,'C', 'D' ,'E', 'F', 'G', 'H', 'I' ,'K', 'L', 'M', 'N' ,'P' ,'Q' ,'R' ,'S', 'T', 'V', 'W' ,'Y']) -set(list(x))
        
        for c in temp:
            x.append(c)
            y.append(0)
        
        _index.append(k)
        _seq.append(list(x))
        _per.append(y)
        

In [ ]:
res=[]
peps=""
count=0
for i in range(data.shape[0]): 
    temp=[ int(float(x)*100)/100 for x in data["values"][i].split("|")]
#     print(min(temp))
    if min(temp)< 70 and (70.06  in temp) :
        count+=1
print(count)


In [ ]:
res=[]
peps=""
count=0
for i in range(data.shape[0]): 
    temp=[ int(float(x)*100)/100 for x in data["values"][i].split("|")]
#     print(min(temp))
    if min(temp)<=70.06 and 70.06 in temp  and ("P" not in data.peptide[i] and  "R" not in data.peptide[i]): #or "N" in data.peptide[i] or "D" in data.peptide[i]): #or "K" in data.peptide[i] ) :
#         print(data.peptide[i])
        peps+="".join(np.unique([ x for x in data.peptide[i]]))
        res+= [ int(float(x)*100)/100 for x in data["values"][i].split("|")]
        count+=1
print(count)

In [ ]:
temp=pd.DataFrame(np.unique(res,return_counts=True)).transpose()
temp.columns=["mz","freq"]
temp=temp[temp.freq>10].reset_index(drop=True)
temp=temp.sort_values(by="freq",ascending=False).reset_index(drop=True)
temp.head(10)

In [ ]:
temp=pd.DataFrame(np.unique([x for x in peps],return_counts=True)).transpose()
temp.columns=["mz","freq"]
temp=temp[temp.freq>10].reset_index(drop=True)
temp=temp.sort_values(by="freq",ascending=False).reset_index(drop=True)
temp.head(20)

In [ ]:
res=[]
peps=""
_pepts=""
count=0
for i in range(data.shape[0]):
    temp=[ int(float(x)*10)/10 for x in data["values"][i].split("|")]
    if ((70.0 in temp) and ("P"  not  in data.peptide[i].strip()) ):
#         print(data.peptide[i])
        peps+="".join(np.unique([ x for x in data.peptide[i]]))
        _pepts+=","+data.peptide[i]
        res+= [ int(float(x)*100)/100 for x in data["values"][i].split("|")]
        count+=1
print(count)

In [ ]:
temp=pd.DataFrame(np.unique(res,return_counts=True)).transpose()
temp.columns=["mz","freq"]
temp=temp[temp.freq>10].reset_index(drop=True)
temp=temp.sort_values(by="freq",ascending=False).reset_index(drop=True)
temp.head(10)

In [ ]:
temp=pd.DataFrame(np.unique([x for x in peps],return_counts=True)).transpose()
temp.columns=["mz","freq"]
temp=temp[temp.freq>10].reset_index(drop=True)
temp=temp.sort_values(by="freq",ascending=False).reset_index(drop=True)
temp.head(20)

In [ ]:
res=[]
for i in [x for x in _pepts.split(',') if len(x)>0]:
#     print(i)
#     print(list(combinations(i, 2)))
    for j in range(1,len(i)):
        res.append(i[j-1:j+1])
#         print(i[j-1:j+1])

In [ ]:
temp=pd.DataFrame(np.unique(res,return_counts=True)).transpose()
temp.columns=["aa","counts"]
temp=temp.sort_values(by="counts",ascending=False).reset_index(drop=True)
temp.head(20)

In [ ]:
res=[]
peps=""
count=0
for i in range(data.shape[0]):
    temp=[ int(float(x)*10)/10 for x in data["values"][i].split("|")]
    if ((70.0 in temp) and (72.0 in temp) and min(temp)<=70 ):# and ("P"  not  in data.peptide[i].strip()) ):
        peps+="".join(np.unique([ x for x in data.peptide[i]]))
        res+= [ int(float(x)*100)/100 for x in data["values"][i].split("|")]
        if (("I"    in data.peptide[i].strip()) or
           ("L"    in data.peptide[i].strip()) or
           ("V"    in data.peptide[i].strip())):
            count+=1
print(count)

In [ ]:
775/815

In [ ]:
temp=pd.DataFrame(np.unique([x for x in peps],return_counts=True)).transpose()
temp.columns=["mz","freq"]
temp=temp[temp.freq>10].reset_index(drop=True)
temp=temp.sort_values(by="freq",ascending=False).reset_index(drop=True)
temp.head(20)

In [ ]:
[x.strip() for x in sorted(['do-2-10pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_008.mzML    '
,'do-2-12p5pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_009.mzid    '
,'do-2-12p5pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_009.mzML  '
,'do-2-15pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_010.mzid      '
,'do-2-15pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_010.mzML    '
,'do-2-17p5pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_011.mzid    '
,'do-2-17p5pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_011.mzML  '
,'do-2-20pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_012.mzid      '
,'do-2-20pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_012.mzML    '
,'do-2-50pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_013.mzid      '
,'do-2-50pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_013.mzML    '
,'do-2-5pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_006.mzid       '
,'do-2-5pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_006.mzML     '
,'do-2-7p5pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_007.mzid    '
,'do-2-7p5pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_007.mzML   ']) if ".mzid" in x]

In [ ]:
[x.strip().replace('.mzML','.csv') for x in sorted(['do-2-10pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_008.mzML    '
,'do-2-12p5pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_009.mzid    '
,'do-2-12p5pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_009.mzML  '
,'do-2-15pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_010.mzid      '
,'do-2-15pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_010.mzML    '
,'do-2-17p5pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_011.mzid    '
,'do-2-17p5pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_011.mzML  '
,'do-2-20pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_012.mzid      '
,'do-2-20pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_012.mzML    '
,'do-2-50pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_013.mzid      '
,'do-2-50pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_013.mzML    '
,'do-2-5pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_006.mzid       '
,'do-2-5pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_006.mzML     '
,'do-2-7p5pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_007.mzid    '
,'do-2-7p5pct_90mingrad_60OTit_1p3s_cycletime_E_wr_03Feb23_007.mzML   ']) if ".mzML" in x]